In [8]:
import pandas as pd
import numpy as np
import pickle

In [9]:
# 1638 words
with open("../Data/wordpools/wasnorm_wordpool.txt") as f:
    items = f.readlines()
items = [i.replace("\n","") for i in items]
items = np.array(items)

In [10]:
simu_sess_num = 1000
n = 25
v = 5
n_exitem = 15
n_expair = 15
extra = 2 * (n_exitem * 1 + n_expair * 2)
wordpool = np.arange(1,1639)

In [11]:
df_study = pd.DataFrame()
df_test = pd.DataFrame()
rng = np.random.default_rng(seed=42)

for sess in range(simu_sess_num):
    tmp_df_study = pd.DataFrame()
    tmp_df_test1 = pd.DataFrame()
    tmp_df_test2 = pd.DataFrame()
    all_words = rng.choice(wordpool, 2*(n+v) + extra, replace=False)

    # study
    pres_words = all_words[0:2*n]
    pres_words = np.reshape(pres_words, (n,2))
    tmp_df_study['study_itemno1'] = pres_words[:,0]
    tmp_df_study['study_itemno2'] = pres_words[:,1]
    tmp_df_study['study_item1'] = items[pres_words[:,0]-1]
    tmp_df_study['study_item2'] = items[pres_words[:,1]-1]
    tmp_df_study['session'] = sess
    tmp_df_study['pair_idx'] = np.arange(n)
    # df_study = pd.concat([df_study,tmp_df_study])

    # repeat 6 times, no continuous
    idx_order = []
    repeat_cnt = np.zeros(24)
    number_left = np.arange(24)
    last_pick = 0
    for i in range(24*6):
        pick = rng.choice(number_left)

        loop_cnt = 0
        while pick == last_pick:
            pick = rng.choice(number_left)
            loop_cnt += 1
            if loop_cnt > 100:
                print("fail")
                break

        idx_order.append(pick)
        repeat_cnt[pick] += 1
        if repeat_cnt[pick] == 6:
            np.delete(number_left, pick)
        last_pick = pick
    tmp_df_study = tmp_df_study.iloc[idx_order]
    df_study = pd.concat([df_study,tmp_df_study])

    # test conditions
    valid_words = all_words[0:2*(n+v)]
    valid_words = np.reshape(valid_words, (n+v,2))
    conditions = ["Same Item"] * 6 + ["Different Item"] * 6 + ["Intact Pair"] * 3 + ["Item-Pair"] * 9 + ["Pair-Item"] * 6
    conditions = rng.permutation(conditions)

    test1_probe = []
    test2_probe = []
    for i, c in enumerate(conditions):
        order = rng.choice([0,1])
        this_pair = valid_words[i].tolist()
        if c == "Same Item":
            test1_probe.append([this_pair[order],-1])
            test2_probe.append([this_pair[order],-1])
        elif c == "Different Item":
            test1_probe.append([this_pair[order],-1])
            test2_probe.append([this_pair[1-order],-1])
        elif c == "Intact Pair":
            test1_probe.append(this_pair)
            test2_probe.append(this_pair)
        elif c == "Item-Pair":
            test1_probe.append([this_pair[order],-1])
            test2_probe.append(this_pair)
        elif c == "Pair-Item":
            test1_probe.append(this_pair)
            test2_probe.append([this_pair[order],-1])
    test1_probe = np.array(test1_probe)
    test2_probe = np.array(test2_probe)

    # test1
    extra_words = all_words[2*(n+v):2*(n+v)+int(extra/2)]
    extra_items = extra_words[0:n_exitem]
    extra_pairs = np.reshape(extra_words[n_exitem:], (n_expair,2))
    tmp_df_test1['test_itemno1'] = np.concatenate((test1_probe[:,0], extra_items, extra_pairs[:,0]))
    tmp_df_test1['test_itemno2'] = np.concatenate((test1_probe[:,1], [-1]*n_exitem, extra_pairs[:,1]))
    tmp_df_test1['test_item1'] = items[tmp_df_test1['test_itemno1']-1]
    tmp_df_test1['test_item2'] = items[tmp_df_test1['test_itemno2']-1]
    tmp_df_test1['correct_ans'] = [1] * n + [0] * (v + n_exitem + n_expair)
    tmp_df_test1['pair_idx'] = np.concatenate((np.arange(n+v),[-1] * (n_exitem + n_expair)))
    tmp_df_test1['type'] =  np.concatenate((conditions, ["extra"] * (n_exitem + n_expair)))
    tmp_df_test1['test'] = 1
    tmp_df_test1['session'] = sess
    tmp_df_test1 = tmp_df_test1.iloc[rng.permutation(tmp_df_test1.index)].reset_index(drop=True)

    # test2
    extra_words = all_words[-int(extra/2):]
    extra_items = extra_words[0:n_exitem]
    extra_pairs = np.reshape(extra_words[n_exitem:], (n_expair,2))
    tmp_df_test2['test_itemno1'] = np.concatenate((test2_probe[:,0], extra_items, extra_pairs[:,0]))
    tmp_df_test2['test_itemno2'] = np.concatenate((test2_probe[:,1], [-1]*n_exitem, extra_pairs[:,1]))
    tmp_df_test2['test_item1'] = items[tmp_df_test2['test_itemno1']-1]
    tmp_df_test2['test_item2'] = items[tmp_df_test2['test_itemno2']-1]
    tmp_df_test2['correct_ans'] = [1] * n + [0] * (v + n_exitem + n_expair)
    tmp_df_test2['pair_idx'] = np.concatenate((np.arange(n+v),[-1] * (n_exitem + n_expair)))
    tmp_df_test2['type'] = np.concatenate((conditions, ["extra"] * (n_exitem + n_expair)))
    tmp_df_test2['test'] = 2
    tmp_df_test2['session'] = sess
    tmp_df_test2 = tmp_df_test2.iloc[rng.permutation(tmp_df_test2.index)].reset_index(drop=True)

    df_test = pd.concat([df_test,tmp_df_test1,tmp_df_test2])

df_study.reset_index(inplace = True, drop=True)
df_test.reset_index(inplace = True, drop=True)  

In [12]:
df_study.iloc[30:40]

,study_itemno1,study_itemno2,study_item1,study_item2,session,pair_idx
30,897,346,MINK,COLLEGE,0,19
31,1490,1062,TOWER,PIE,0,11
32,897,346,MINK,COLLEGE,0,19
33,517,1270,EDITORIAL,SERPENT,0,23
34,1079,1346,PIZZA,SPAGHETTI,0,7
35,1027,773,PATROL,KEY,0,4
36,1572,685,VIRUS,HANDCUFFS,0,22
37,736,1155,INDIAN,RACKET,0,15
38,1601,1409,WHALE,SUPERMARKET,0,6
39,133,1198,BLACKBOARD,ROBE,0,14


In [13]:
df_test.iloc[10:40]

,test_itemno1,test_itemno2,test_item1,test_item2,correct_ans,pair_idx,type,test,session
10,1245,-1,SCARECROW,ZOO,0,-1,extra,1,0
11,346,-1,COLLEGE,ZOO,1,19,Different Item,1,0
12,1535,-1,UMPIRE,ZOO,0,-1,extra,1,0
13,185,577,BUCKLE,FLEA,0,-1,extra,1,0
14,1027,773,PATROL,KEY,1,4,Intact Pair,1,0
15,760,966,JOURNAL,OAR,0,-1,extra,1,0
16,566,-1,FIN,ZOO,1,12,Different Item,1,0
17,790,203,LABEL,BUTCHER,0,-1,extra,1,0
18,1324,-1,SLEEVE,ZOO,1,20,Different Item,1,0
19,639,-1,GLASS,ZOO,1,16,Same Item,1,0


In [14]:
# save
with open('../Data/simuS2_design.pkl', 'wb') as outp:
    pickle.dump(df_study, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(df_test, outp, pickle.HIGHEST_PROTOCOL)